<a href="https://colab.research.google.com/github/Sohammhatre10/Predicto/blob/main/NewsAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


In [3]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

In [4]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at human-centered-summarization/financial-summarization-pegasus and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
monitored_ticker = ['BTC', 'DOGE', 'LTE', 'ETH', 'XRP']

In [92]:
def search_for_crypto_news(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&sca_esv=554280007&rlz=1C1ONGR_enIN1063IN1063&tbs=sbd:1,qdr:h&tbm=nws&sxsrf=AB5stBje0RC3zr3lO-wUJpYaxs5vVrh8bA:1691350266356&source=lnt&sa=X&ved=2ahUKEwjJ-cPe4siAAxVKe2wGHX-vDnYQpwV6BAgCEAc&biw=1280&bih=595".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

In [87]:
raw_urls = {}
for ticker in monitored_ticker:
  raw_urls[ticker] = search_for_crypto_news(ticker)

In [88]:
import re

In [89]:
exclude_list = ['maps', 'policies','preferences', 'accounts', 'support']

In [90]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [91]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_ticker}
cleaned_urls

{'BTC': ['https://www.google.com/search?q%3Dyahoo%2Bfinance%2BBTC%26sca_esv%3D554280007%26rlz%3D1C1ONGR_enIN1063IN1063%26tbs%3Dsbd:1,qdr:h%26tbm%3Dnws%26sxsrf%3DAB5stBje0RC3zr3lO-wUJpYaxs5vVrh8bA:1691350266356%26source%3Dlnt%26sa%3DX%26ved%3D2ahUKEwjJ-cPe4siAAxVKe2wGHX-vDnYQpwV6BAgCEAc%26biw%3D1280%26bih%3D595%26pccc%3D1'],
 'DOGE': ['https://www.google.com/search?q%3Dyahoo%2Bfinance%2BDOGE%26sca_esv%3D554280007%26rlz%3D1C1ONGR_enIN1063IN1063%26tbs%3Dsbd:1,qdr:h%26tbm%3Dnws%26sxsrf%3DAB5stBje0RC3zr3lO-wUJpYaxs5vVrh8bA:1691350266356%26source%3Dlnt%26sa%3DX%26ved%3D2ahUKEwjJ-cPe4siAAxVKe2wGHX-vDnYQpwV6BAgCEAc%26biw%3D1280%26bih%3D595%26pccc%3D1'],
 'LTE': ['https://www.google.com/search?q%3Dyahoo%2Bfinance%2BLTE%26sca_esv%3D554280007%26rlz%3D1C1ONGR_enIN1063IN1063%26tbs%3Dsbd:1,qdr:h%26tbm%3Dnws%26sxsrf%3DAB5stBje0RC3zr3lO-wUJpYaxs5vVrh8bA:1691350266356%26source%3Dlnt%26sa%3DX%26ved%3D2ahUKEwjJ-cPe4siAAxVKe2wGHX-vDnYQpwV6BAgCEAc%26biw%3D1280%26bih%3D595%26pccc%3D1'],
 'ETH': ['https://ww

In [93]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [94]:
for ticker in monitored_ticker:
    if ticker=='BTC':
      btc = articles["BTC"]
      eth = articles["ETH"]
      lte = articles["LTE"]
      xrp = articles["XRP"]
      doge = articles["DOGE"]


In [98]:
def summarize(articles):
    summaries = []
    for ticker, article_list in articles.items():
        article_text = '\n\n'.join(article_list)  # Join the list of articles into a single string
        ticker_summaries = []
        for chunk in article_text.split('\n\n'):  # Split the article into paragraphs (you can adjust this based on your article structure)
            input_ids = tokenizer.encode(chunk, return_tensors='pt', max_length=512, truncation=True)
            output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
            summary = tokenizer.decode(output[0], skip_special_tokens=True)
            ticker_summaries.append(summary)
        summaries.append({ticker: ticker_summaries})
    return summaries

# Assuming you have a dictionary with lists of articles for each ticker (e.g., btc, eth, doge, xrp, lte)
articles = {
    'BTC': btc,
    'ETH': eth,
    'DOGE': doge,
    'XRP': xrp,
    'LTE': lte
}

# Call the summarize function with the articles dictionary
summaries = summarize(articles)
print(summaries)

[{'BTC': ['HIVE Blockchain, Block, and CME Group are among top crypto stocks.', 'V2 expands list of tradable assets to include dogecoin. Initial liquidity pools pay out as much as 45% for DOGE', 'ADNOC to own 35% of Absheron gas field after completion of transaction. TotalEnergies has been present in Azerbaijan since 1996', 'Miami Mayor has long been an advocate for digital assets. Suarez says he would ban central bank digital currency', 'All images are copyrighted.', 'Multi-currency, crypto-based platform is breaking down barriers to financial inclusion.', 'Google Bard says Sunday is best day to buy crypto. Bitcoin price averaged lower from Saturdays through Mondays over last year of trading', 'All photographs subject to copyright.', 'SQ is one of the largest crypto stocks in the past three months.', '‘No matter what your view on ESG, demonstrating how Bitcoin makes positive contributions’ says ESG analyst.', 'Mining operations are located in the United States, Canada, and Iceland.']}

In [107]:
from transformers import pipeline

# Initialize the sentiment analysis pipeline
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model_revision = "af0f99b"
sentiment_classifier = pipeline('sentiment-analysis', model=model_name, revision=model_revision)

def analyze_sentiment(summaries):
    sentiment_results = sentiment_classifier(summaries)  # Use the pipeline directly on the list of summaries
    return sentiment_results

# Assuming you want to analyze the sentiment for the 'BTC' ticker's summaries
btc_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('BTC', [])]
eth_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('ETH', [])]  # List of summaries for ETH
doge_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('DOGE', [])]  # List of summaries for DOGE
xrp_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('XRP', [])]  # List of summaries for XRP
lte_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('LTE', [])]  # List of summaries for LTE

sentiment_results_btc = analyze_sentiment(btc_summaries)
sentiment_results_eth = analyze_sentiment(eth_summaries)
sentiment_results_doge = analyze_sentiment(doge_summaries)
sentiment_results_xrp = analyze_sentiment(xrp_summaries)
sentiment_results_lte = analyze_sentiment(lte_summaries)


data = {
    'Ticker': ['BTC'] * len(sentiment_results_btc) + ['ETH'] * len(sentiment_results_eth) +
              ['DOGE'] * len(sentiment_results_doge) + ['XRP'] * len(sentiment_results_xrp) +
              ['LTE'] * len(sentiment_results_lte),
    'Result_state': [result['label'] for result in sentiment_results_btc] +
                       [result['label'] for result in sentiment_results_eth] +
                       [result['label'] for result in sentiment_results_doge] +
                       [result['label'] for result in sentiment_results_xrp] +
                       [result['label'] for result in sentiment_results_lte],
    'Score': [result['score'] for result in sentiment_results_btc] +
                       [result['score'] for result in sentiment_results_eth] +
                       [result['score'] for result in sentiment_results_doge] +
                       [result['score'] for result in sentiment_results_xrp] +
                       [result['score'] for result in sentiment_results_lte],
}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('sentiment_analysis_results.csv', index=False)